In [1]:
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from speakerfeatures import get_MFCC
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import time
import os
from collections import Counter
from sklearn.svm import SVC

In [2]:
source   = "development_set\\"
dest = "speaker_models\\"
train_file = "development_set_enroll_5.txt" # "train.txt" # 
n_files = 5 #training_sample

In [3]:
count = 1
file_paths = open(train_file,'r')

label_dict = {}
y_train = []
x_train = np.asarray(())
features = np.asarray(())
avg_data = 0
training_data = 0

for i, path in enumerate(file_paths):    
    path = path.strip()   
    #print(path)
    Fs, audio = read(source + path)
    vector = get_MFCC(audio, Fs, 20)
    training_data += len(audio)
    
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
    #print(len(features))
    # when features of 5 files of speaker are concatenated, then do model training
    if count == n_files:    
        if x_train.size == 0:
            x_train = features
        else:
            x_train = np.vstack((x_train, features))
        for j in range(len(features)):
            y_train.append((i+1)//n_files)
        print()
        mn = features.shape
        print((i+1)//n_files, 'Modeled :',path.split("-")[0], ", Training Data:", training_data/Fs, ", MFCC:", mn[1])
        label_dict[(i+1)//n_files] = path.split("-")[0]
        features = np.asarray(())
        avg_data += training_data
        training_data = 0
        count = 0
    count = count + 1
file_paths.close()
print("Avg training_data:", avg_data/(Fs*75))


1 Modeled : akshay_hindi , Training Data: 19.969160997732427 , MFCC: 40

2 Modeled : aman_maghi , Training Data: 8.4649375 , MFCC: 40

3 Modeled : aman_realtime , Training Data: 19.969160997732427 , MFCC: 40

4 Modeled : aman , Training Data: 28.16458049886621 , MFCC: 40

5 Modeled : ankit_maithli , Training Data: 19.969160997732427 , MFCC: 40

6 Modeled : premananda_manipur , Training Data: 19.969160997732427 , MFCC: 40

7 Modeled : sumit_up , Training Data: 19.969160997732427 , MFCC: 40

8 Modeled : anthonyschaller , Training Data: 16.0 , MFCC: 40

9 Modeled : Apple_Eater , Training Data: 20.8213125 , MFCC: 40

10 Modeled : Ara , Training Data: 40.5 , MFCC: 40

11 Modeled : argail , Training Data: 24.75 , MFCC: 40

12 Modeled : ariyan , Training Data: 25.125 , MFCC: 40

13 Modeled : arjuan , Training Data: 27.125 , MFCC: 40

14 Modeled : armond , Training Data: 53.37225 , MFCC: 40

15 Modeled : Artem , Training Data: 28.0 , MFCC: 40

16 Modeled : arthur , Training Data: 37.875 , MFC

In [4]:
print("Avg training_data:", avg_data/(Fs*75))

Avg training_data: 29.041700833333334


In [5]:
label_dict

{1: 'akshay_hindi',
 2: 'aman_maghi',
 3: 'aman_realtime',
 4: 'aman',
 5: 'ankit_maithli',
 6: 'premananda_manipur',
 7: 'sumit_up',
 8: 'anthonyschaller',
 9: 'Apple_Eater',
 10: 'Ara',
 11: 'argail',
 12: 'ariyan',
 13: 'arjuan',
 14: 'armond',
 15: 'Artem',
 16: 'arthur',
 17: 'artk',
 18: 'arun',
 19: 'arvala',
 20: 'asalkeld',
 21: 'asladic',
 22: 'AslakKnutsen',
 23: 'asp',
 24: 'AT',
 25: 'atamur',
 26: 'ataru80',
 27: 'atterer',
 28: 'audioodyssey',
 29: 'avsa242',
 30: 'ax',
 31: 'axllaruse',
 32: 'azmisov',
 33: 'B',
 34: 'bachroxx',
 35: 'bae',
 36: 'Bahoke',
 37: 'Bareford',
 38: 'bart',
 39: 'Bassel',
 40: 'bdutta',
 41: 'beady',
 42: 'bebe',
 43: 'becz03',
 44: 'beez1717',
 45: 'belmontguy',
 46: 'ben',
 47: 'bencoder',
 48: 'bendauphinee',
 49: 'bendra',
 50: 'bennmann',
 51: 'benob',
 52: 'benoliver999',
 53: 'Berrym',
 54: 'BFG',
 55: 'bhart',
 56: 'bhavanikrishna',
 57: 'BhushanNKIITBombay',
 58: 'bhuvan',
 59: 'bigmbywater',
 60: 'BigMick',
 61: 'BillMorgan',
 62: '

In [6]:
len(x_train), len(x_train[0]), y_train[0], label_dict[y_train[0]]

(311446, 40, 1, 'akshay_hindi')

In [7]:
len(x_train), len(y_train)

(311446, 311446)

In [ ]:
model = SVC(C=1, class_weight = "balanced", decision_function_shape = 'ovo')
model.fit(x_train, y_train)

In [ ]:
picklefile = "model_5_rbf.svm"
cPickle.dump(model, open(dest + picklefile,'wb'))

# TEST

In [ ]:
test_file = "development_set_test.txt"  # "test.txt" #   
file_paths = open(test_file, 'r')

svm_model = cPickle.load(open(dest + "model_5_rbf.svm",'rb'))

files  = [os.path.join(source, f[0:len(f)-1]) for f in file_paths]

In [ ]:
len(files)

In [ ]:
files[0].split("\\")[1].split("-")[0]

In [ ]:
accuracy = 0
total_sample = 0
for i, path in enumerate(files):
    total_sample += 1
    Fs, audio = read(path)
    vector = get_MFCC(audio, Fs, 20)
    temp = svm_model.predict(vector)
    y_test = Counter(temp).most_common()[0]
    if(path.split("\\")[1].split("-")[0] == label_dict[y_test[0]]):
        accuracy += 1
        print(i+1, ":", path.split("\\")[1].split("-")[0], "Predicted as:", label_dict[y_test[0]])
    else:
        print("-----------------------------------------------")
        print(i+1, ":", path.split("\\")[1].split("-")[0], "Predicted as:", label_dict[y_test[0]])
        print("-----------------------------------------------")
print("Accuracy =", accuracy/total_sample*100, "%")

Accuracy = 99.75124378109453 %, C=1, class_weight = "balanced", MFCC = 20, training data = 3 files

Accuracy = 95.86776859504133 %, C=1, class_weight = "balanced", MFCC = 13

Accuracy = 95.86776859504133 %, C=1, class_weight = "balanced", decision_function_shape = 'ovo', MFCC = 13

Accuracy = 100.0 %SVC(C=1, class_weight = "balanced", decision_function_shape = 'ovo')

In [ ]:
Fs, audio = read("development_set\\Apple_Eater-20091026-yul\\wav\\a0061.wav")
vector = get_MFCC(audio, Fs, 20)
temp = svm_model.predict(vector)
plt.plot(temp)
from collections import Counter m
y_test = Counter(temp).most_common()[0]
print("Predicted as:", label_dict[y_test[0]])

In [ ]:
Counter(temp).most_common()

In [ ]:
label_dict[1]

In [ ]:
file_paths.close()

In [ ]:
plt.plot(vector[0])

In [ ]:
path

In [ ]:
204/3